In [3]:
import datrie
import string
import sklearn.cluster
import pandas as pd
import numpy as np
import numba as nb
import os
import warnings

from numba import njit, jit
from numba.typed import Dict, List
from numba import types
from openpyxl import Workbook
from collections import namedtuple

from myLevenshtein import Levenshtein_modified

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# ==== 3.1 Tag Name / Value to Tag Master Name / Value Mapping ====

- Load all dataset
- Simple data pre-processing

In [179]:
raw_data = []
for dirpath, dirnames, filenames in os.walk('data/Azure'):
    for filename in filenames:
        df = pd.read_csv(dirpath + '/' + filename, low_memory=False, header=1)
        df = df[['Date', 'AccountId', 'AccountName', 'DepartmentId', 'DepartmentName', 'InstanceId', 'ResourceGroup', 'Tags']]
        df['ResourceName'] = df['InstanceId'].str.split('/').apply(lambda x: x[-1])
        raw_data.append(df.drop(columns=['InstanceId']))
raw_data = pd.concat(raw_data)
data = raw_data.dropna(subset=['Tags', 'ResourceGroup']).reset_index()

- Parse Tags using `numba` to speed up, time-consuming < 4s, which takes around 30s without `numba`
- Filter Tag which has no Tag Value

In [198]:
numba_dict = Dict.empty(key_type=types.int64, value_type=types.UniTuple(types.string, 2))


@jit(nopython=True, nogil=True, parallel=True)
def get_all_tags_typed_dict(all_tags_list, numba_dict):
    index = np.int64(0)
    for tags in all_tags_list:
        tags = tags.replace('"', '')
        tags = tags[3:-1].split(',  ')
        for tag in tags:
            tmp = tag.split(': ')
            if tmp[1] != '':
                numba_dict[index] = (tmp[0], tmp[1])
                index += 1

    return numba_dict


all_tags_typed_dict = get_all_tags_typed_dict(data['Tags'].tolist(), numba_dict)

- Transfer to pandas

In [181]:
all_tags_df = pd.DataFrame.from_dict(all_tags_typed_dict, orient='index', columns=['Tag Name', 'Tag Value'])

- Define method to get Tag Master Names / Tag Master Values  
*Hint:* Use `Affinity Propagation (AP)` to choose Master Names / Values automatically, this method doesn't need to identify the number of clusters

In [182]:
def get_master(words, **kwargs):
    damping = kwargs.get('damping', 0.5)
    max_iter = kwargs.get('max_iter', 200)
    convergence_iter = kwargs.get('convergence_iter', 15)
    similarity = kwargs.get('similarity', None)
    affinity = 'euclidean' if similarity is None else 'precomputed'
    preference = kwargs.get('preference', None)

    if affinity == 'euclidean':
        ap = sklearn.cluster.AffinityPropagation(affinity=affinity,
                                                 damping=damping,
                                                 max_iter=max_iter,
                                                 convergence_iter=convergence_iter).fit(words)
    else:
        preference = np.median(similarity) if preference is None else preference
        ap = sklearn.cluster.AffinityPropagation(affinity=affinity,
                                                 damping=damping,
                                                 preference=preference,
                                                 max_iter=max_iter,
                                                 convergence_iter=convergence_iter).fit(similarity)

    master = {}
    index = 0
    for cluster_id in np.unique(ap.labels_):
        exemplar = words[ap.cluster_centers_indices_[cluster_id]]
        cluster = np.unique(words[np.nonzero(ap.labels_ == cluster_id)])
        cluster_str = ", ".join(cluster)
        print(" - *%s:* %s" % (exemplar, cluster_str))
        for point in cluster:
            master[index] = {'0': point, '1': exemplar}
            index += 1
        #     print(Levenshtein_modified(point, exemplar), end=', ')
        # print()
    return master

- Get Tag Master Names
- Store result in csv to quickly call and check them conveniently later

In [183]:
tag_names = all_tags_df.groupby('Tag Name')['Tag Value'].count().to_dict()
words = np.asarray(list(tag_names.keys()))
similarity = [[-Levenshtein_modified(w1, w2, True) for w2 in words] for w1 in words]
tmp = list(tag_names.values())
tmp_sum = np.sum(tmp)
preference = [-(1 - i / tmp_sum)*3 for i in tmp]

master_names_dict = get_master(words, similarity=similarity, preference=preference)
master_names_df = pd.DataFrame.from_dict(master_names_dict, orient='index')
master_names_df.columns = ['Tag Name', 'Tag Master Name']
master_names_df.to_csv('./result/Tag Master Names.csv', sep=',', header=True, index=False)

 - *Client:* Client, client, x_Client
 - *ClusterId:* ClusterId
 - *ClusterName:* ClusterName
 - *Creator:* Creator, DatabricksInstancePoolCreatorId, x_Creator
 - *DataFactoryEntityResourceId:* DataFactoryEntityResourceId
 - *Databricks-ElasticDisk:* Databricks-ElasticDisk
 - *DatabricksInstanceGroupId:* DatabricksInstanceGroupId
 - *DatabricksInstancePoolId:* DatabricksInstancePoolId
 - *Environment:* DatabricksEnvironment, Enviornment, Environment, databricks-environment, environment
 - *JobId:* JobId
 - *MappingDataflowRunId:* MappingDataflowRunId
 - *Product:* Product
 - *ResourceClass:* ResourceClass
 - *RunName:* RunName
 - *Service:* Service, x_Service
 - *SqlEndpointId:* SqlEndpointId
 - *Type:* Type, type, x_Type
 - *Vendor:* Vendor
 - *application:* application
 - *createdBy:* createdBy
 - *databricks-instance-name:* databricks-instance-name
 - *dbsql-channel:* dbsql-channel
 - *hidden-link:/subscriptions/f6650bec-0252-45ce-b7ce-8fff251c4dad/resourcegroups/OldMutual/providers

- Get Tag Master Values for some Tag Master Names, which can be changed

In [185]:
master_names = ['Client', 'Type']
master_values_df = []
for master_name in master_names:
    tag_values = all_tags_df[all_tags_df['Tag Name'].str.contains('|'.join(master_names_df[master_names_df['Tag Master Name'] == master_name]['Tag Name']))]
    tag_values = tag_values.groupby('Tag Value')['Tag Name'].count()
    tag_values = tag_values[tag_values>5].to_dict()

    words = np.asarray(list(tag_values.keys()))
    similarity = [[-Levenshtein_modified(w1, w2, True) for w2 in words] for w1 in words]
    tmp = list(tag_values.values())
    tmp_sum = np.sum(tmp)
    preference = [-(1 - i / tmp_sum) for i in tmp]

    master_values_dict = get_master(words, similarity=similarity, preference=preference)
    tmp = pd.DataFrame.from_dict(master_values_dict, orient='index')
    tmp.columns = ['Tag Value', 'Tag Master Value']
    tmp['Tag Master Name'] = [master_name] * len(tmp)
    master_values_df.append(tmp)

master_values_df = pd.concat(master_values_df, ignore_index=True)
master_values_df.to_csv('./result/Tag Master Values.csv', sep=',', header=True, index=False)

 - *ARM:* ARM
 - *Absa:* ABSA, ABSACF, ABSACLOUD313, ABSACLOUDQA, ABSARBB, Absa
 - *Alexander Forbes:* Alexander Forbes
 - *Blue Label Telecoms:* Blue Label Telecoms
 - *CCBA:* CCBA, CCBACloud, ccba
 - *Carlisle:* Carlisle
 - *Discovery:* Discovery
 - *Edcon:* Edcon
 - *Ellix:* Ellix
 - *Exxaro:* Exxaro
 - *HCC:* HC, HCC
 - *Internal:* Internal, Internal - Cloud, Internal - Multi-Dim, Internal - Scenarios
 - *Internal - Pentest:* Internal - Pentest, Pentest
 - *Investec:* Investec
 - *KMOD:* KMOD
 - *Liberty:* Liberty
 - *Massmart:* Massmart
 - *NTT:* NTT
 - *National Life:* National Life
 - *OECD:* OECD, oecd
 - *Old Mutual:* OLD MUTUAL, Old Mutual, OldMutual
 - *QAmultidim:* QA multidim hybrid, QAmultidim, qa multidim hybrid, qamultidim
 - *RMB:* RMB
 - *SANDBOX:* SANDBOX
 - *SPAR:* SPAR, SPARCLOUD, Spar, sparcloud
 - *Sanlam:* Sanlam
 - *Shared:* Shared, Shared Services
 - *Stanlib:* STANLIB, Stanlib, stanlib
 - *Suncorp:* Suncorp
 - *TFG:* TFG
 - *Vitality:* Vitality
 - *pb3Test:* 

- Match Tag Name to Tag Master Name & give matching percentage
- Store Tag Master Value in Trie to quickly find whether one string exsits, whose time complexity is *O(n)*

In [186]:
trie1 = datrie.Trie(string.ascii_letters + string.digits)
master_names_df = pd.read_csv('./result/Tag Master Names.csv', low_memory=False)
master_names = master_names_df['Tag Master Name'].unique()
for i in range(len(master_names)):
    trie1[''.join(filter(str.isalnum, str(master_names[i]).lower()))] = i
trie1.save('./result/Tag Master Name Trie.txt')

trie2 = datrie.Trie(string.ascii_letters + string.digits)
master_values_df = pd.read_csv('./result/Tag Master Values.csv', low_memory=False)
master_values = master_values_df['Tag Master Value'].unique()
for i in range(len(master_values)):
    trie2[''.join(filter(str.isalnum, str(master_values[i]).lower()))] = i
trie2.save('./result/Tag Master Value Trie.txt')

In [187]:
trie = datrie.Trie.load('./result/Tag Master Name Trie.txt')
tag = 'DatabrickInstace'
tag = ''.join(filter(str.isalnum, tag.lower()))
test_list = [tag[i:i + 4] for i in range(len(tag))]

suggestion = []
for t in test_list:
    res = trie.items(t)
    for r in res:
        master = master_names[r[1]]
        distance = Levenshtein_modified(tag, master)
        percentage = (1 - distance / max(len(master), len(tag))) * 100
        if distance != 1000:
            suggestion.append([master, distance, f'{percentage:.2f}%'])
suggestion = np.array(suggestion)
suggestion = suggestion[np.argsort(suggestion[:, 2])][::-1]
suggestion

array([['databricks-instance-name', '6.0', '75.00%'],
       ['DatabricksInstancePoolId', '8.0', '66.67%'],
       ['DatabricksInstanceGroupId', '9.0', '64.00%'],
       ['Databricks-ElasticDisk', '9.0', '59.09%']], dtype='<U32')

In [188]:
trie = datrie.Trie.load('./result/Tag Master Value Trie.txt')
tag = 'waweabsacfadmin'
tag = ''.join(filter(str.isalnum, tag.lower()))
test_list = [tag[i:i + 4] for i in range(len(tag))]

suggestion = []
for t in test_list:
    res = trie.items(t)
    for r in res:
        master = master_values[r[1]]
        distance = Levenshtein_modified(tag, master)
        percentage = (1 - distance / max(len(master), len(tag))) * 100
        if distance != 1000:
            suggestion.append([master, distance, f'{percentage:.2f}%'])
suggestion = np.array(suggestion)
suggestion = suggestion[np.argsort(suggestion[:, 2])][::-1]
suggestion

array([['Absa', '11.0', '26.67%']], dtype='<U32')

# ==== 3.2 MO Code & CC Code Matching ====

- Load dataset
- Parse "cloud_tags" to extract "resourcetags_user_costcenter" & "resourcetags_user_appid"

In [12]:
# AWS raw data is too large and we only need the column of 'cloud_tags'
data_aws = pd.read_excel('./data/Sample Data for AWS Private and Confidential.xlsx', sheet_name='Tags Extracted')
data_aws = data_aws.dropna().reset_index()

data_aws_mo = pd.read_excel('./data/Sample Data for AWS Private and Confidential.xlsx', sheet_name='measured object hierarchy - 202')
data_aws_mo = data_aws_mo.dropna(subset=['MeasuredObjectCode']).reset_index()

data_aws_cc = pd.read_excel('./data/Sample Data for AWS Private and Confidential.xlsx', sheet_name='cost centre hierarchy - 2022062')
data_aws_cc = data_aws_cc.dropna(subset=['CostCentreCode']).reset_index()

In [ ]:
@jit(nopython=True, nogil=True, parallel=True)
def get_costcenter_appid(all_tags_list):
    index = np.int64(0)
    costercenter, appid = [], []
    for tags in all_tags_list:
        tags = (tags.replace('"', '')).replace('\\', '')
        tags = tags[1:-1].split(', ')
        for tag in tags:
            tmp = tag.split(' : ')
            if tmp[0] == 'resourcetags_user_costcenter':
                costercenter.append(tmp[1])
            if tmp[0] == 'resourcetags_user_appid':
                appid.append(tmp[1])

    return costercenter, appid

costercenter, appid = get_costcenter_appid(data_aws['cloud_tags'].to_list())
costercenter, appid = np.unique(costercenter), np.unique(appid)

- Store Measured Object Code & Cost Centre Code in Trie

In [ ]:
mo = data_aws_mo['MeasuredObjectCode'].astype(str).unique()
cc = data_aws_cc['CostCentreCode'].astype(str).unique()

# 10: MeasuredObjectCode
# 01: CostCentreCode
# 11: is both MeasuredObjectCode and CostCentreCode
trie = datrie.Trie(string.ascii_letters + string.digits + ' -_')
for m in mo:
    trie[m] = 10
for c in cc:
    trie[c] = 11 if c in trie else 1

- Check each 'resourcetags_user_costcenter' & 'resourcetags_user_appid' value
- Store results in xlsx

In [ ]:
def mo_cc_match(test_values, test_name, writer, sheet_name):
    res = {}
    index = 0
    for c in test_values:
        tmp = trie[str(c)] if str(c) in trie else 0
        match1 = c if tmp // 10 else 'No'
        match2 = c if tmp % 10 else 'No'
        res[index] = (c, match1, match2)
        index += 1
    res = pd.DataFrame(res).T
    res.columns = [test_name, 'MO Code Match', 'CC Code Match']
    res.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
file_name = './result/MO_CC Code Match.xlsx'
if not os.path.exists(file_name):
    Workbook().save(file_name)
writer = pd.ExcelWriter(file_name, engine='openpyxl', mode='w')
mo_cc_match(costercenter, 'resourcetags_user_costcenter', writer, 'user_costcenter')
mo_cc_match(appid, 'resourcetags_user_appid', writer, 'user_appid')
writer.save()

# ==== 3.3 Unique Tag Value Check ====

In [189]:
data_unique = pd.read_excel('./data/3.3 MagicOrange Unique Tag Value Use Case Sample Set.xlsx', sheet_name='3.3 Unstructured Results')

- Delete rows where all both 'ResourceGroup' and 'Resource' are missing
- Filter duplicated data

In [190]:
data_unique.dropna(how='all', inplace=True)
data_unique.drop_duplicates(inplace=True)


- Check the resource number if unique and there are no items missing in sequence or in the run

In [191]:
df = data_unique.copy()

# find ResourceGroup with no ResourceName
tmp = df[df['Resource'].isnull()]
print(f'Warning: ResourceGroup {tmp["ResourceGroup"].to_list()} need to be assigned with ResourceName')

# find ResourceName with no ResourceGroup mapped
tmp = df[df['ResourceGroup'].isnull()]
print(f'Warning: ResourceName {tmp["Resource"].to_list()} need to be mapped to ResourceGroup')

# find ResourceGroup are assigned with more than one ResourceName
tmp = []
for key, group in df.groupby('ResourceGroup'):
    if len(group)>1:
        tmp.append(key)
if len(tmp):
    print(f'Warning: ResourceGroup {tmp} are assigned with more than one ResourceName')

# find ResourceName are mapped to more than one ResourceGroup
tmp = []
for key, group in df.groupby('Resource'):
    if len(group) > 1:
        tmp.append(key)
if len(tmp):
    print(f'Warning: ResourceName {tmp} are mapped to more than one ResourceGroup')


In [192]:
# check the ResourceName sequence
# Resource-001
# last three digits are the sequence number
df = data_unique.copy()
df = df.dropna(subset=['Resource'])
df['Resource'] = df['Resource'].apply(lambda x: int(x[-3:]) if isinstance(x, str) and len(x) > 3 and x[-3:].isdigit() else -1000)

tmp = df[df['Resource'] == -1000]
if len(tmp):
    print(f'Warning: please check whether the ResourceName for ResourceGroup {tmp["ResourceGroup"].to_list()} is in right format')

tmp = df[df['Resource'] != -1000]
tmp = df.sort_values(by='Resource', ascending=True)
if tmp.loc[0]['Resource'] != 1:
    print('Warning: the ResourceName sequence is not begin with 1')
tmp = tmp.drop_duplicates(subset=['Resource']).reset_index()
continuous_check = tmp['Resource'] - tmp['Resource'].shift()
continuous_check = continuous_check[continuous_check > 1]
if len(continuous_check):
    print('Warning: there is no ResourceName in periods:', end=' ')
    for i in continuous_check.keys():
        print(f'[{tmp.loc[i-1]["Resource"]+1}, {tmp.loc[i]["Resource"]-1}]', end=' ')

# ==== 3.4 Tag Suggestions ====

In [193]:
data_suggest = pd.read_excel('./data/Resource Group Resource Name Tag Value Matches Azure Data Sample 20062022.xlsx')

In [196]:
master_values_df = pd.read_csv('./result/Tag Master Values.csv')

def get_suggestion(test_resource_group, test_resource_name, master_values):
    result = {}
    def my_filter(char):
        if char in string.ascii_letters + string.digits:
            return True
        return False
    for i in range(len(test_resource_group)):
        tmp1, tmp2 = '', ''
        for m in master_values:
            master_value = ''.join(filter(my_filter, m.lower()))
            group = ''.join(filter(my_filter, test_resource_group[i].lower()))
            name = ''.join(filter(my_filter, test_resource_name[i].lower()))
            match1 = (master_value in group or group in master_value)
            match2 = (master_value in name or name in master_value)
            if match1 & match2:
                tmp1 = tmp2 = m
            elif match1:
                tmp1 = m
            elif match2:
                tmp2 = m
        result[i] = (tmp1, tmp2)
    return result


suggestion_dict = get_suggestion(data_suggest['ResourceGroup'].astype(str).tolist(), data_suggest['Resource Name'].astype(str).tolist(),
                                 master_values_df['Tag Master Value'].astype(str).tolist())
suggestion_df = pd.DataFrame.from_dict(suggestion_dict, orient='index')
suggestion_df.columns = ['Tag Master Value based on ResourceGroup Match', 'Tag Master Value based on ResourceName Match']
suggestion_df = pd.concat([data_suggest.loc[:, ['ResourceGroup', 'Resource Name']], suggestion_df], axis=1)
suggestion_df.to_csv('./result/Suggestion.csv', index=False, header=True)
suggestion_df

,ResourceGroup,Resource Name,Tag Master Value based on ResourceGroup Match,Tag Master Value based on ResourceName Match
0,HCC,asausprodhcc,HCC,HCC
1,Shared-Services,storausdeploytest,Shared,
2,Shared-Services,Tenant,Shared,
3,shared-services,aspausiso,Shared,
4,Test,storwederan,pb3Test,
...,...,...,...,...
4404,absa,aspweg1,Absa,
4405,absa,waweabsacloudqa,Absa,Absa
4406,absa,waweinvestecadmin,Absa,Investec
4407,absa,waweinvestecprod,Absa,Investec
